<br></br>

<div align="center">

<h1 align="center">
    Cuaderno para el trabajo de clasificación relacional
    <br></br>
    Estudio de métricas de clustering
</h1>

<h6 align="center">
    Antonio Macías Ferrera (antmacfer1@alum.us.es)
    <br></br>
    Delfín Santana Rubio (delsanrub@alum.us.es)
    <br></br>
    Universidad de Sevilla
</h6>

<br></br>

## **Control de Versiones**
    
| **Fecha**  | **Versión** | **Descripción**               |
| :--------- | :---------- | :---------------------------- |
| 27/05/2024 | v1r0        | Primera versión del cuaderno. |
| 29/05/2024 | v1r1        | Refactorización e inclsuión de otras medidas de centralidad.      |
| 07/06/2024 | v1r2        | Correcciones de formato.      |


</div>

<br></br>

## **Índice de contenido**

1. [Introducción](#introducción)
2. [Instanciación del grafo](#creacion)
3. [Implementación de funciones](#creacion)
4. [Estudio de las métricas](#estudio)
    - [Coeficiente de clustering](#clustering)
    - [Coeficiente de squares clustering](#squares-clustering)

<br></br>

# <a name="introducción"></a> 1. **Introducción**

En este cuadernillo se encuentra todo el código utilizado para comprobar métricas de clustering. Se han implementado dos funciones que calculan la métrica proporcinada para cada tipo de datos, y otra que evalúa la métrica haciendo uso del t-test Student.

<br></br>

# <a name="creacion"></a> 2. **Instanciación del grafo**

<br></br>

In [4]:
import matplotlib.pyplot as plt
import networkx as nx
import json
import csv

# Step 1: Read the edges.csv file
with open('../facebook_large/musae_facebook_edges.csv', 'r') as f:
    edges_data = csv.reader(f)
    next(edges_data,None)
    edges = [tuple(row) for row in edges_data]

# Step 3: Create a NetworkX graph object
G = nx.Graph()

# Step 4: Add nodes and edges to the graph
G.add_edges_from(edges)

tvshow = []
government = []
politician = []
company = []

# Step 5: Assign features to nodes or edges
with open('../facebook_large/musae_facebook_target.csv', 'r') as f:
    edges_data = csv.reader(f)
    next(edges_data,None)
    for row in edges_data:
        target = row[-1]
        node = row[0]
        if target == "government":
            government.append(node)
        elif target == "tvshow":
            tvshow.append(node)
        elif target == "politician":
            politician.append(node)
        elif target == "company":
            company.append(node)
            
        G.nodes[node]["target"] = target


# <a name="creacion"></a> 3. **Implementación de funciones**

La función ```calc_metrics(results_dict)``` calcula la medida de centralidad dada para cada conjunto de datos del mismo tipo. Para cada nodo, se obtiene la métrica y se realiza la media y la desviación típica por tipo (). Así, con la desviación típica podemos ver si una medida es relativamente coherente en cada grupo. Si una métrica tiene una desviación típica pequeña en un grupo, quiere decir que esa métrica es similar para ese grupo, por lo que puede servir para diferenciarlo; mientras que si la desviación típica es grande, querrá decir que esa métrica es muy dispar dentro de ese grupo, por lo que no arrojará un resultado concluyente.

Por otro lado, la función ```t_Student_test(results_dict)``` genera un t-test Student que compara una métrica dada de cada grupo con el resto de grupos. Esto se hace para poder evaluar si una métrica es suficientemente diferente entre los distintos grupos para poder ser usada como atributo de entrenamiento del modelo. 

In [5]:
from numpy import std
from numpy import average

def calc_metrics(results_dict):
    tvshow_results = [results_dict[x] for x in tvshow]
    government_results = [results_dict[x] for x in government]
    politician_results = [results_dict[x] for x in politician]
    company_results = [results_dict[x] for x in company]

    tvshow_avg = average(tvshow_results)
    tvshow_deviation = std(tvshow_results)
    print("TVShow average= ",tvshow_avg," , deviation= ",tvshow_deviation)

    government_avg = average(government_results)
    government_deviation = std(government_results)
    print("Government average= ",government_avg," , deviation= ",government_deviation)

    politician_avg = average(politician_results)
    politician_deviation = std(politician_results)
    print("Politician average= ",politician_avg," , deviation= ",politician_deviation)

    company_avg = average(company_results)
    company_deviation = std(company_results)
    print("Company average= ",company_avg," , deviation= ",company_deviation)

from scipy.stats import ttest_ind
from itertools import combinations

# Calcular el test de t de Student
def t_Student_test(results_dict):
    tvshow_results = [results_dict[x] for x in tvshow]
    government_results = [results_dict[x] for x in government]
    politician_results = [results_dict[x] for x in politician]
    company_results = [results_dict[x] for x in company]
    datasets = [tvshow_results, government_results, politician_results, company_results]
    res = {}

    for (i, data_i), (j, data_j) in combinations(enumerate(datasets), 2):
        t_stat, p_value = ttest_ind(data_i, data_j)
        res[f"data{i+1} vs data{j+1}"] = (t_stat, p_value)

    for key, value in res.items():
        print(f"{key}: T-statistic = {value[0]}, P-value = {value[1]}")


# <a name="estudio"></a> 4. **Estudio de las métricas**

En este apartado se evaluará cada métrica para hacer una selección 'a priori'. Se pretende descartar métricas que aporten muy mal resultado en el Student test. Se considera mal resultado si el p-value es mayor a 0.05.

### <a name="clustering"></a> **Coeficiente de clustering**


In [6]:
# Coeficiente de clustering
clustering = nx.clustering(G)
calc_metrics(clustering)
t_Student_test(clustering)

TVShow average=  0.3524535354313019  , deviation=  0.3540146469057918
Government average=  0.41713675511342435  , deviation=  0.2711972573411726
Politician average=  0.39216743742579085  , deviation=  0.3029095117118667
Company average=  0.27387003646729674  , deviation=  0.3389491926519668
data1 vs data2: T-statistic = -10.18534462019756, P-value = 3.010702747554816e-24
data1 vs data3: T-statistic = -5.655130229691308, P-value = 1.6043659430754746e-08
data1 vs data4: T-statistic = 10.709900635502233, P-value = 1.2825756411998836e-26
data2 vs data3: T-statistic = 4.888290247638987, P-value = 1.0296431964189525e-06
data2 vs data4: T-statistic = 27.062014321785295, P-value = 4.507779980271968e-157
data3 vs data4: T-statistic = 20.272827013869268, P-value = 6.598624181664373e-90


### <a name="squares-clustering"></a> **Coeficiente de squares clustering**


In [7]:
# Coeficiente de squares clustering ( no parece dar un buen resultado)
clustering = nx.square_clustering(G)
calc_metrics(clustering)
t_Student_test(clustering)

TVShow average=  0.13955281977191394  , deviation=  0.19072525217794437
Government average=  0.13873400472771732  , deviation=  0.11928419000763953
Politician average=  0.13637561364702372  , deviation=  0.1301550749643918
Company average=  0.100813244285221  , deviation=  0.16991846089255488
data1 vs data2: T-statistic = 0.2647428248590908, P-value = 0.7912129314159827
data1 vs data3: T-statistic = 0.9409732209025197, P-value = 0.34674357915272636
data1 vs data4: T-statistic = 10.250962791309476, P-value = 1.5571949933807112e-24
data2 vs data3: T-statistic = 1.062179263778633, P-value = 0.2881745951599348
data2 vs data4: T-statistic = 15.003302849043617, P-value = 1.797359370475241e-50
data3 vs data4: T-statistic = 12.887071949680916, P-value = 9.356156153445658e-38


Observamos que el coeficiente de clustering aporta muy buen resultado mientras que el squares clustering solo aporta un buen resultado en el 50% de los casos